In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report
import sys
np.set_printoptions(threshold=sys.maxsize)
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

In [18]:
df = pd.read_csv("data/application_train.csv")
df_test = pd.read_csv("data/application_test.csv")

In [19]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# get rid of object features
dtypes = df.dtypes
dtypes = dtypes[dtypes!='object']
features = list(set(dtypes.index)-set(['TARGET']))
len(features)

105

In [21]:
X = df[features]
y = df['TARGET']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
model = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,
                  n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=16, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=16,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=40, reg_lambda=40, scale_pos_weight=4, seed=None,
              silent=None, subsample=1, verbosity=1)

In [26]:
from sklearn.metrics import roc_auc_score
y_train_predicted = model.predict_proba(X_train)[:,1]
y_test_predicted = model.predict_proba(X_test)[:,1]
print("Train AUC %.4f"%roc_auc_score(y_train,y_train_predicted))
print("Test AUC %.4f"%roc_auc_score(y_test,y_test_predicted))

Train AUC 0.7746
Test AUC 0.7540


In [27]:
def generate_results(model,df_test,features,id_col,target,file):
    dft = df_test[features]
    results = df_test[[id_col]]
    results[target] = model.predict_proba(dft)[:,1]
    results.to_csv(file,index=False,columns=results.columns)
    
generate_results(model,df_test,features,"SK_ID_CURR","TARGET","results/results.csv")

/home/peyman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
%cd  results
!kaggle  competitions  submit -c home-credit-default-risk -f results.csv -m "Xgb model EDA FE with numeric cols"
%cd ..

In [ ]:
!kaggle competitions submissions -c home-credit-default-risk -q